# Scrapping data

In [2]:
import numpy as np
import pandas as pd
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup
from pandas import Series
import requests as r, json
import re


START_URL='https://auto.ru/moskva/cars/used/'

In [3]:
pages_url_list=[]

for i in range(1, 200):
    response = r.get( START_URL+'?page=%s'%i)
    page = BeautifulSoup(response.text, 'html.parser')
    for link in page.findAll('a',{"class": "Link ListingItemTitle__link"}):
        try:
            pages_url_list.append(link['href'])
        except:
            pass

/var/folders/8t/ljy40y7s6_q7mblbcyxxb2ch0000gn/T/ipykernel_87768/864436625.py:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for link in page.findAll('a',{"class": "Link ListingItemTitle__link"}):


In [ ]:
cars_list = []

for item in pages_url_list:
  response = r.get(item)
  response.encoding = 'utf8'
  page = BeautifulSoup(response.text, 'html.parser')
  price = page.find('span', 'OfferPriceCaption__price')
  data = page.find('ul', 'CardInfo__list-MZpc1')
  car_info = {}
  if data and price:
    price = price.string
    price_number = re.sub(r'\s+', '', price).replace('₽', '')
    car_info['price'] = price_number
    for row in data.find_all('li', class_='CardInfoRow'):
      key = row.find('div', class_='CardInfoRow__cell').text.strip()
      value = row.find_all('div', class_='CardInfoRow__cell')[1].text.strip()
      car_info[key] = re.sub(r'\s+', '', value)
    cars_list.append(car_info)


df = pd.DataFrame(cars_list)


In [ ]:
df

,price,Наличие,Поколение,Год выпуска,Пробег,Кузов,Цвет,Двигатель,Комплектация,Налог,...,Состояние,Владельцы,ПТС,Таможня,Обмен,Владение,Запас хода,VIN,Госномер,Гарантия
0,5750000,Вналичии,IIРестайлинг,2024,10км,внедорожник5дв.,белый,2.0л/150л.с./Бензин,High,5250₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,Рассмотрюварианты,NaN,NaN,NaN,NaN,NaN
1,48800000,Впути,III(W447)Рестайлинг2,2024,25км,минивэн,чёрный,2.0л/237л.с./Дизель,8опций,17700₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,NaN,9месяцев,NaN,NaN,NaN,NaN
2,46900000,Вналичии,III(W447)Рестайлинг2,2023,25км,минивэн,чёрный,2.0л/237л.с./Дизель,9опций,17775₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,NaN,9месяцев,NaN,NaN,NaN,NaN
3,499000,Вналичии,I,2003,277000км,внедорожник5дв.,чёрный,2.2л/114л.с./Дизель,44опции,3192₽/год,...,Нетребуетремонта,2владельца,Оригинал,Растаможен,Рассмотрюварианты,NaN,NaN,NaN,NaN,NaN
4,5900000,Назаказ,NaN,2025,1км,седан,серый,673л.с./495кВт/Электро,Max,0₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,NaN,NaN,800км,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6953,550000,Вналичии,NaN,1994,339280км,седан,синий,2.3л/225л.с./Бензин,1опция,16200₽/год,...,Нетребуетремонта,3илиболее,Оригинал,Растаможен,NaN,NaN,NaN,YV1**************,******|797,NaN
6954,16900000,Вналичии,NaN,2024,11км,купе,чёрный,3.0л/449л.с./Бензин,AMGCLE534matic+,67350₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN
6955,2290000,Вналичии,III(F55/F56)Рестайлинг,2018,106000км,хэтчбек3дв.,синий,2.0л/192л.с./Бензин,CooperS,9600₽/год,...,Нетребуетремонта,2владельца,Оригинал,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN
6956,20900000,Вналичии,II(V167)Рестайлинг,2024,68км,внедорожник5дв.,белый,4.0л/612л.с./Бензин,GLE63S4MATIC+,275400₽/год,...,Нетребуетремонта,1владелец,Оригинал,Растаможен,NaN,2месяца,NaN,NaN,NaN,NaN
